In [6]:
from coscine import CoscineClient  # available via `pip install coscine`
from coscine.exceptions import RequirementError
import ipywidgets as widgets

class CoscineGUI:
    def __init__(self):
        self._output = widgets.Output()
        self._body_box = widgets.VBox()
        self._box = widgets.HBox([self._body_box, self._output])
        self._client = None
        self._submit_button = widgets.Button(description='Submit')
        self._storage = {}

        self._init_client_box()

    def _init_client_box(self):
        self._submit_button.on_click(self._register_client)
        self._storage['token_w'] = widgets.Password(description="API Token")
        self._storage['path_w'] = widgets.FileUpload(description="Upload Token")
        children = [widgets.HTML('<h2>Connect to CoScInE</h2>'),
                    widgets.HTML('Please provide the <a href=https://help.itc.rwth-aachen.de/en/service/b2b7729fd93f4c7080b475776f6b5d87/article/2fc14c42d0784ecfae0a797bd5e3754e/> '
                                 'API token </a> directly OR chose a file where it is stored:'),
                    self._storage['token_w'],
                    self._storage['path_w'],
                    self._submit_button]
        
        self._body_box.children = tuple(children)
    
    def _register_client(self, _button=None):
        self._output.clear_output()
        with self._output:
            print("")
        token = self._storage['token_w'].value
        path = self._storage['path_w'].value
        if len(token)>0 and len(path)>0:
            with self._output:
                print("Provide API token OR file path!")
            self._init_client_box()
        elif len(token)>0:
            self._client = CoscineClient(token=token, verbose=False)
        elif len(path)>0:
            token = list(path.values())[0]['content'].decode('utf8')
            self._client = CoscineClient(token=token, verbose=False)
        else:
            with self._output:
                print("No token provided!")
        
        if self._client is not None:
            try:
                self._body_box.children = tuple([widgets.HTML("Connecting...")])
                self._get_all_projects()
            except HTTPError as e:
                with self._output:
                    print(f"Error: {e.args[0]}")
                self._client = None
                self._init_client_box()
            else:
                del self._storage['token_w']
                del self._storage['path_w']
                self._init_resources()
                self._init_select_resource_gui()
        
    def _get_all_projects(self):
        self._storage['projects'] = self._client.get_projects()
    
    def _init_resources(self):
        resources = {}
        project = self._storage['projects'][0]
        resources[project['slug']] = self._client.get_resources(project)
        self._storage['resources'] = resources
    
    def _print_project_info(self, pr_id):
        for key in ['projectName', 'displayName', 'description', 'keywords', 'slug']:
            print(f"     {key}:  {self._storage['projects'][pr_id][key]}")
    
    def _on_project_change(self, change):
        new_pr_id = change['new']
        self._storage['resource_w'].disable = True
        self._output.clear_output()
        with self._output:
            self._print_project_info(new_pr_id)
        self._storage['resource_w'] = self._gen_resource_dropbox(new_pr_id)
        self._select_resource_gui()
        
    def _gen_resource_dropbox(self, project_id):
        pr = self._storage['projects'][project_id]
        slug = pr['slug']
        self._storage['resources'][slug] = self._client.get_resources(pr)
        res_names = [(res["displayName"], i) for i, res in enumerate(self._storage['resources'][slug])]
        if len(res_names)>0:
            return widgets.Dropdown(description="Resource", options=res_names, value=0)
        else:
            return widgets.Dropdown(description='Resource', options=res_names)
    
    def _select_resource_gui(self, _button=None):
        self._body_box.children = tuple([
            self._storage['project_w'],
            self._storage['resource_w'],
            self._submit_button
        ])
        self._box = widgets.HBox([self._body_box, self._output])

    def _init_select_resource_gui(self):
        self._submit_button = widgets.Button(description="Submit")
        self._submit_button.on_click(self._init_upload_gui)
        pr_names = [(pr['projectName'], i) for i, pr in enumerate(self._storage['projects'])]
        pr_w = widgets.Dropdown(description="Project", options=pr_names, value=0)
        pr_w.observe(self._on_project_change, names='value')
        self._storage['project_w'] = pr_w
        self._storage['resource_w'] = self._gen_resource_dropbox(0)
        self._select_resource_gui()
    
    def _change_metadata(self, change):
        new_val = change['new']
        key = change['owner'].description
        self._storage['MetaDataForm'][key] = new_val
    
    def _init_upload_gui(self, _button=None):
        self._output.clear_output()
        res_idx = self._storage['resource_w'].value
        if res_idx is None:
            with self._output:
                print("No resource selected!")
            return
        self._submit_button = widgets.Button(description="Upload")
        self._submit_button.on_click(self._upload_data)
        back_button = widgets.Button(description="Back")
        back_button.on_click(self._select_resource_gui)
        pr_idx = self._storage['project_w'].value
        res_idx = self._storage['resource_w'].value
        pr = self._storage['projects'][pr_idx]
        res = self._storage['resources'][pr['slug']][res_idx]
        self._storage["choosen_resource"] = res
        form = self._client.MetadataForm(pr, res)
        self._storage['MetaDataForm'] = form
        
        self._storage['file_w'] = widgets.Text(description="Path to file")
        
        info_w = widgets.HTML(f"<h2> Upload data </h2> <p> Upload data to Resource <it>{res['displayName']}</it> of"
                              f" Project <it>{pr['projectName']}</it>: </p>")
        
        children = [widgets.HTML("<h3>Metadata</h3>")]
        for key in form.keys():
            if form.is_controlled(key):
                vocab = form.get_vocabulary(key).keys()
                if len(vocab) > 9:
                    wid = widgets.Combobox(description=key, options=vocab, ensure_option=True)
                else:
                    wid = widgets.Dropdown(description=key, options=vocab)
            else:
                wid = widgets.Text(description=key)
            wid.observe(self._change_metadata, names='value')
            children.append(wid)
        self._storage["MetaData_w"] = widgets.VBox(children)
        
        self._body_box.children = tuple([widgets.HBox([
            widgets.VBox([info_w, self._storage['file_w'], 
                          widgets.HBox([self._submit_button, back_button]),
                          self._output]),
            self._storage['MetaData_w']
            ])])
        
        self._box.children = tuple([self._body_box])
        
    def _upload_data(self, _button=None):
        self._output.clear_output()
        file = self._storage['file_w'].value
        filename = os.path.basename(file)
        if not os.path.isfile(file):
            with self._output:
                print("Error: {file} is not a file!")
            return
        form = self._storage['MetaDataForm']
        res = self._storage["choosen_resource"]
        files_in_res = [file["Name"] for file in self._client.get_files(res)]
        if filename in files_in_res:
            with self._output:
                print(f"Error: {filename} already present in resource.")
            return
        try:
            form_str = form.generate()
        except RequirementError as e:
            with self._output:
                print(e.args[0])
        else:
            self._client.verbose = True
            with self._output:
                self._client.upload_file(res, filename, file, form_str)
            self._client.verbose = False
    
    @property
    def _widget(self):
        return(self._box)
    
    def _ipython_display_(self):
        display(self._widget)

In [7]:
CoscineGUI()